In [2]:
import time
import requests
import json
import pandas as pd

In [3]:
df = pd.read_csv('processed_to_geo.csv', low_memory=False)
columns = ['No_voie', 'Type_de_voie', 'Voie', 'Code_postal', 'Commune']

if 'adresse' not in df:
    df['adresse'] = df[columns].apply(lambda col: ' '.join(col.values.astype(str)), axis = 1)

df.set_index('adresse', inplace=True)
for col in ['longitude', 'latitude']:
    if col not in df:
        df[col] = ''

print(df.head())

,Unnamed: 0,Id,Date_mutation,Valeur_fonciere,No_voie,Type_de_voie,Voie,Code_postal,Commune,Code_departement,Type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,Prix_par_m_bati,Prix_par_m_terrain,longitude,latitude
adresse,,,,,,,,,,,,,,,,,,
26 RUE DE MONTHOLON 1000 BOURG-EN-BRESSE,0,0,2019-04-01,37220,26,RUE,DE MONTHOLON,1000,BOURG-EN-BRESSE,1,Appartement,20.0,1.0,NaN,1861.000000,0.000000,,
22 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE,1,1,2019-04-01,185100,22,RUE,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,1,Appartement,62.0,3.0,NaN,2985.483871,0.000000,,
22 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE,2,2,2019-04-01,185100,22,RUE,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,1,Dépendance,0.0,0.0,NaN,0.000000,0.000000,,
3 RUE DES CHAMPAGNES 1160 PRIAY,3,3,2019-08-01,209000,3,RUE,DES CHAMPAGNES,1160,PRIAY,1,Maison,90.0,4.0,940.0,2322.222222,222.340426,,
5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,4,4,2019-07-01,134900,5,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,1,Maison,101.0,5.0,490.0,1335.643564,275.306122,,


In [4]:
def adresse_to_lon_lat(adresse):
    url = "https://api-adresse.data.gouv.fr/search"
    response = requests.get(url, params = {'q': adresse, 'limit': '1'})
    result = response.json()
    longitude = result['features'][0]['geometry']['coordinates'][0]
    latitude = result['features'][0]['geometry']['coordinates'][1]
    return (longitude, latitude)

In [5]:
df_empty = df
df_empty = df_empty[df_empty['longitude'] == '']
df_empty = df_empty[df_empty['latitude'] == '']

while not df_empty.empty:
    adresses = pd.Series(df_empty.index)
    adresses.drop_duplicates(inplace=True)

    t = time.time()
    for adresse in adresses[0:100]:
        lon, lat = adresse_to_lon_lat(adresse)
        df['longitude'][adresse] = lon
        df['latitude'][adresse] = lat
        df_empty.drop(index=adresse, inplace=True)

    print(time.time()-t, "secondes")

    df.to_csv('processed_to_geo.csv')

    print("shape of df_empty remaining ", df_empty.shape)

KeyboardInterrupt: 